In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import html5lib
import re
import pandas as pd
import csv
import warnings
import xlsxwriter
warnings.filterwarnings('ignore')

In [2]:
def parse_PIK(URL):
    URL = URL
    session = requests.session()
    session.headers = {
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
          "Accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
      }
    try:

        res= session.get(URL)
        res.raise_for_status()

        html = res.text 
        soup = BeautifulSoup(html, 'lxml')


        
        
        price_txt = soup.select_one('div.sc-gyKfoT').text
        price = (int(re.sub('\D', '', price_txt)))

        name_s = soup.select('a.styles__Item-f1ro62-1')
        name = name_s[-1].text

        flat = soup.select_one('h1.sc-iVxNgP').text
        flat_type = flat.split()[0]
        square = float(re.sub(',', '.', flat.split()[1]))

        booked_s=soup.select_one('div.sc-hogtv egyukJ')
        booked = booked_s.text if booked_s else 'No'         

        row =     [{
        'Дата': pd.Timestamp.today(),    
        'ЖК': name,
        'цена': price,
        'Тип квартиры': flat_type,
        'Площадь': square,
        'Стоимость за кв': round(price/square)
        }]

    except Exception as ex:
         row = [{
            'Дата': pd.Timestamp.today(),    
            'ЖК': 'Продано',
            'цена': '',
            'Тип квартиры': '',
            'Площадь': '',
            'Стоимость за кв':'',
            'Забронирована': ''
                }]

    
    return row

In [3]:
def parse_RG(URL):
    URL = URL
    session = requests.session()
    session.headers = {
          'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
          "Accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
      }
    try:

        res= session.get(URL)
        res.raise_for_status()

        html = res.text 
        soup = BeautifulSoup(html, 'lxml')

    
        price_txt = soup.select_one('div.flat-aside__promo-price').text
        price = (int(re.sub('\D', '', price_txt)))

        name_s = soup.select_one('a.flat-aside__complex-link')
        name = name_s.text.lstrip().rstrip()

        flat = soup.select_one('h1.flat-hero__mobile-title').text
        flat_type = flat.split()[0]
        square = float(flat.split()[1])

        booked_s=soup.select_one('div.flat-hero__mobile-booked')
        booked = booked_s.text.lstrip().rstrip() if booked_s else 'No' 

        row = [{
        'Дата': pd.Timestamp.today(),    
        'ЖК': name,
        'цена': price,
        'Тип квартиры': flat_type,
        'Площадь': square,
        'Стоимость за кв': round(price/square),
        'Забронирована': booked
        }]
    
    except Exception as ex:
         row = [{
            'Дата': pd.Timestamp.today(),    
            'ЖК': 'Продано',
            'цена': '',
            'Тип квартиры': '',
            'Площадь': '',
            'Стоимость за кв':'',
            'Забронирована': ''
                }]
    
    return row

In [4]:
df = pd.DataFrame([])
df=df.append(parse_PIK('https://www.pik.ru/flat/749761'), ignore_index=True)
df=df.append(parse_PIK('https://www.pik.ru/flat/810349'), ignore_index=True)
df=df.append(parse_PIK('https://www.pik.ru/flat/809272'), ignore_index=True)
df=df.append(parse_RG('https://rg-dev.ru/novostroyki/oktyabrskoe-pole/32-1-korpus/1-k_kvartira-studiya_27m_OKP-1-22-266/'), ignore_index=True)
df=df.append(parse_RG('https://rg-dev.ru/novostroyki/oktyabrskoe-pole/32-1-korpus/1-k_kvartira-studiya_27m_OKP-1-21-255/'), ignore_index=True)
df=df.append(parse_RG('https://rg-dev.ru/novostroyki/oktyabrskoe-pole/32-1-korpus/1-k_kvartira-studiya_23m_OKP-1-19-224/'), ignore_index=True)

data = pd.read_excel('flats.xlsx')
data=data.append(df, ignore_index=True)
data.to_excel('flats.xlsx', index=False)